In [7]:
from fastai.vision.all import *

In [8]:
#Imports

import pandas as pd
import os
import numpy as np
from PIL import Image
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import random
import torch
from torchvision.models import inception_v3


In [9]:
import torch
print(f"Using CUDA: {torch.cuda.is_available()}")


Using CUDA: True


In [10]:
# Reading in the training data
csv_path = os.path.join('COMP90086_2024_Project_train', 'train.csv')
train_data = pd.read_csv(csv_path)
train_dir = os.path.join('COMP90086_2024_Project_train', 'train')
print(train_data.head())


    id  shapeset  type  total_height  instability_type  cam_angle  \
0   54         2     1             3                 1          1   
1  173         1     1             4                 1          2   
2  245         1     1             4                 1          2   
3  465         2     1             5                 0          1   
4  611         2     1             3                 1          1   

   stable_height  
0              2  
1              1  
2              1  
3              5  
4              1  


In [11]:
# Define the path to your CSV file
csv_path = os.path.join('COMP90086_2024_Project_train', 'train.csv')

# Read the CSV file into a DataFrame
train_data = pd.read_csv(csv_path)

# Adding a column with the complete image path
train_data['image_path'] = train_data['id'].apply(lambda x: os.path.join('COMP90086_2024_Project_train', 'train', f"{x}.jpg"))

# Ensure that the 'stable_height' column contains categorical data for classification
train_data['stable_height'] = train_data['stable_height'].astype(str)  # Convert targets to string if they are numerical

# Convert your DataFrame to a FastAI DataBlock for classification
dblock = DataBlock(
    blocks=(ImageBlock, CategoryBlock),  # Image as input and categorical target
    get_x=ColReader('image_path'),       # Path to the image
    get_y=ColReader('stable_height'),    # Target column
    splitter=RandomSplitter(valid_pct=0.2)  # Splitting the data into training and validation sets
)

# Create a DataLoader
dls = dblock.dataloaders(train_data, bs=8, num_workers=0)  # Adjust batch size as needed

# Create a FastAI learner using ResNet50 for classification
learn = vision_learner(dls, resnet50, metrics=accuracy, loss_func=CrossEntropyLossFlat())

# Train the model using fine-tuning
learn.fine_tune(5)  # Adjust the number of epochs as needed

# Save the trained model
learn.save('resnet50_trained_model')


    id  shapeset  type  total_height  instability_type  cam_angle  \
0   54         2     1             3                 1          1   
1  173         1     1             4                 1          2   
2  245         1     1             4                 1          2   
3  465         2     1             5                 0          1   
4  611         2     1             3                 1          1   

   stable_height                                  image_path  
0              2   COMP90086_2024_Project_train\train\54.jpg  
1              1  COMP90086_2024_Project_train\train\173.jpg  
2              1  COMP90086_2024_Project_train\train\245.jpg  
3              5  COMP90086_2024_Project_train\train\465.jpg  
4              1  COMP90086_2024_Project_train\train\611.jpg  


Downloading: "https://download.pytorch.org/models/resnet50-11ad3fa6.pth" to C:\Users\julev/.cache\torch\hub\checkpoints\resnet50-11ad3fa6.pth
100%|██████████| 97.8M/97.8M [00:09<00:00, 10.9MB/s]


epoch,train_loss,valid_loss,_rmse,time
0,3.435228,2.768990,1.664028,02:13


epoch,train_loss,valid_loss,_rmse,time
0,2.197687,2.058529,1.434758,03:07
1,2.112784,1.741706,1.319737,02:53
2,1.633756,1.618883,1.272353,04:45
3,1.528137,1.468079,1.211643,02:56
4,1.116243,1.541620,1.241620,02:57


Path('models/resnet50_trained_model.pth')

In [25]:
# Define the DataBlock for a classification task
dblock = DataBlock(
    blocks=(ImageBlock, CategoryBlock),  # Image as input and categorical target
    get_x=ColReader('image_path'),       # Path to the image
    get_y=ColReader('stable_height'),    # Column for target labels (ensure they are categorical values)
    splitter=RandomSplitter(valid_pct=0.2), # Splitting the data into training and validation sets
    item_tfms=Resize(299)  # Resize images to 299x299, the required input size for InceptionV3
)

# Create a DataLoader
dls = dblock.dataloaders(train_data, bs=8)  # Adjust the batch size based on your GPU memory

# Modify the InceptionV3 model for classification
class InceptionV3Custom(nn.Module):
    def __init__(self, num_classes):
        super().__init__()
        self.model = inception_v3(weights="DEFAULT")
        self.model.fc = nn.Linear(self.model.fc.in_features, num_classes)  # Adjust final layer for classification

    def forward(self, x):
        if self.training:
            logits, aux_logits = self.model(x)
            return logits
        else:
            return self.model(x)

# Determine the number of classes from the training data
num_classes = len(train_data['stable_height'].unique())

# Initialize the modified InceptionV3 model for classification
inception_model = InceptionV3Custom(num_classes)

# Create a FastAI learner using the modified Inception model
learn = Learner(dls, inception_model, loss_func=CrossEntropyLossFlat(), metrics=accuracy)

# Train the model using fine-tuning
learn.fine_tune(5)  # Adjust the number of epochs as needed

learn.save('inception_v3_trained_model')


epoch,train_loss,valid_loss,accuracy,time


KeyboardInterrupt: 

In [22]:
# Define the path to your test CSV file
test_csv_path = os.path.join('COMP90086_2024_Project_test', 'test.csv')

# Read the CSV file into a DataFrame
test_data = pd.read_csv(test_csv_path)

# Adding a column with the complete image path
test_data['image_path'] = test_data['id'].apply(lambda x: os.path.join('COMP90086_2024_Project_test', 'test', f"{x}.jpg"))

# Create a DataLoader for the test set
test_dls = dls.test_dl(test_data)  # Use test_dl to create a DataLoader for testing



In [23]:
learn = learn.load('inception_v3_trained_model')

In [24]:
# Evaluate the model's performance on the test set
test_loss, test_rmse = learn.validate(dl=test_dls)
print(f"Test Loss: {test_loss}, Test RMSE: {test_rmse}")


Test Loss: None, Test RMSE: None
